In [ ]:
import sys

sys.path.append('..')

: 

In [2]:
import torch

data = torch.load('../processed_data/complete_spectrogram_dataset/processed_data.pt')
label = torch.load('../processed_data/complete_spectrogram_dataset/processed_label.pt')

In [3]:
from audio_toolbox.metrics import audio_dataset_split

device = 'cuda' if torch.cuda.is_available() else 'cpu'
RANDOM_STATE = 42
X_train, y_train, X_val, y_val,\
X_test, y_test = audio_dataset_split(data, label, train_val_test_ratio=(0.9, 0.05, 0.05), random_state=RANDOM_STATE)

In [4]:
import torch
from scipy.sparse import csr_matrix

def sparse_labels_to_tensor(labels):
    scipy_sparse_matrix = csr_matrix(labels)

    # Convert the scipy sparse matrix to a torch sparse tensor
    indices = torch.LongTensor([scipy_sparse_matrix.indices, scipy_sparse_matrix.indptr[:-1]])
    values = torch.from_numpy(scipy_sparse_matrix.data)
    shape = scipy_sparse_matrix.shape

    return torch.sparse_coo_tensor(indices, values, shape).to_dense()

In [5]:
X_train = torch.tensor(X_train, dtype=torch.float).permute(0, 3, 1, 2)
X_val = torch.tensor(X_val, dtype=torch.float).permute(0, 3, 1, 2)
X_test = torch.tensor(X_test, dtype=torch.float).permute(0, 3, 1, 2)
y_train = sparse_labels_to_tensor(y_train)
y_val = sparse_labels_to_tensor(y_val)
y_test = sparse_labels_to_tensor(y_test)

C:\Users\zheng\AppData\Local\Temp\ipykernel_29500\709474325.py:8: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:278.)
  indices = torch.LongTensor([scipy_sparse_matrix.indices, scipy_sparse_matrix.indptr[:-1]])


In [ ]:
from torchvision import transforms

# Define the preprocessing transforms
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [6]:
from torch import nn

model = torch.hub.load('pytorch/vision:v0.9.0', 'vgg16', pretrained=True)
model.features[0] = nn.Conv2d(4, 64, kernel_size=3, padding=1)
num_classes = 10
model.classifier[6] = nn.Linear(4096, num_classes)

Using cache found in C:\Users\zheng/.cache\torch\hub\pytorch_vision_v0.9.0


In [7]:
from torch.utils.data import TensorDataset

datasets = {
    'train': TensorDataset(X_train, y_train),
    'val': TensorDataset(X_val, y_val),
    'test': TensorDataset(X_test, y_test)
}

In [8]:
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from torch import nn
from audio_toolbox.models import OneHotCrossEntropyLoss

loss_fn = OneHotCrossEntropyLoss()
learning_rate = 1e-3  # Adjust the learning rate as needed
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

In [9]:
from audio_toolbox.trainer import ModelTrainer

trainer = ModelTrainer(datasets, model, loss_fn, optimizer, scheduler)

In [10]:
trainer_config = {
    'save': False,
    'num_epochs': 100,
    'batch_size': 32,
    'batch_tqdm': True
}

trainer.train(**trainer_config)

Epoch 0001: 1it [01:43, 103.17s/it]


KeyboardInterrupt: 